# 6. Estrazione dei Landmark per il Dataset ASLLRP con MediaPipe

Questo notebook ha lo scopo di risolvere un problema fondamentale per l'integrazione del dataset ASLLRP nel nostro processo di training: la mancanza dei file di landmark (keypoint) in formato JSON.

Il nostro `LandmarkDataset` si aspetta di trovare questi file pre-generati, ma il dataset ASLLRP fornisce solo i file video.

## Obiettivo

L'obiettivo di questo notebook è automatizzare il processo di estrazione dei landmark dai video di ASLLRP utilizzando **MediaPipe**. I passaggi che seguiremo sono:

1.  **Configurazione dei Percorsi**: Definiremo i percorsi di input (video e file CSV) e di output (dove salvare i file JSON generati).
2.  **Identificazione dei Video**: Leggeremo il file `asllrp_video_sentiment_data_0.25_without_golden.csv` per ottenere la lista esatta dei video da processare.
3.  **Esecuzione di MediaPipe**: Per ogni video, utilizzeremo la libreria MediaPipe per analizzare il video e salvare i landmark (posa, viso, mani) frame per frame.
4.  **Verifica dell'Output**: Alla fine del processo, avremo una nuova cartella contenente i landmark in formato JSON, pronta per essere utilizzata nella fase di addestramento.

**Requisiti:**

- **MediaPipe installato**: È necessario aver installato la libreria con `pip install mediapipe`.
- **OpenCV installato**: Necessario per la lettura dei video (`pip install opencv-python`).

Procediamo con l'implementazione.


In [2]:
import os
import pandas as pd
import cv2
import mediapipe as mp
import json
from tqdm import tqdm

# Ottieni la directory principale del progetto in modo robusto
# Questo notebook si trova in 'notebooks', quindi torniamo indietro di una directory
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

print(f"La directory base del progetto è: {BASE_DIR}")

La directory base del progetto è: /Users/ignazioemanuelepicciche/Documents/TESI Magistrale UCBM/Improved_EmoSign_Thesis


## 1. Configurazione dei Percorsi

In questa sezione, definiamo tutti i percorsi necessari per l'elaborazione. Non sono richieste modifiche manuali in questa cella.


In [3]:
threshold = 0.34

# --- PERCORSI DI INPUT ---


# Percorso della cartella contenente i video di ASLLRP
ASLLRP_VIDEOS_DIR = os.path.join(
    BASE_DIR, "data", "raw", "ASLLRP", "batch_utterance_video_v3_1"
)

# Percorso del file CSV che elenca i video di ASLLRP da usare
ASLLRP_CSV_FILE = os.path.join(
    BASE_DIR,
    "data",
    "processed",
    f"asllrp_video_sentiment_data_{threshold}_without_golden.csv",
)


# --- PERCORSI DI OUTPUT ---

# Cartella principale dove verranno salvati i landmark estratti
OUTPUT_JSON_DIR = os.path.join(
    BASE_DIR, "data", "raw", "ASLLRP", f"mediapipe_output_{threshold}", "json"
)


# --- VERIFICA DEI PERCORSI ---
os.makedirs(OUTPUT_JSON_DIR, exist_ok=True)

print(f"Cartella video ASLLRP: {ASLLRP_VIDEOS_DIR}")
print(f"File CSV ASLLRP: {ASLLRP_CSV_FILE}")
print(f"Cartella output JSON: {OUTPUT_JSON_DIR}")

# Verifica se la cartella dei video esiste
if not os.path.exists(ASLLRP_VIDEOS_DIR):
    print(
        "\nATTENZIONE: Il percorso della cartella dei video di ASLLRP non sembra corretto. Controllalo prima di procedere."
    )

Cartella video ASLLRP: /Users/ignazioemanuelepicciche/Documents/TESI Magistrale UCBM/Improved_EmoSign_Thesis/data/raw/ASLLRP/batch_utterance_video_v3_1
File CSV ASLLRP: /Users/ignazioemanuelepicciche/Documents/TESI Magistrale UCBM/Improved_EmoSign_Thesis/data/processed/asllrp_video_sentiment_data_0.34_without_golden.csv
Cartella output JSON: /Users/ignazioemanuelepicciche/Documents/TESI Magistrale UCBM/Improved_EmoSign_Thesis/data/raw/ASLLRP/mediapipe_output_0.34/json


## 2. Caricamento e Analisi del File CSV

Leggiamo il file CSV per ottenere la lista dei video che dobbiamo processare. Questo ci assicura di elaborare solo i file pertinenti al nostro dataset di training.


In [4]:
# Leggi il file CSV
try:
    df_asllrp = pd.read_csv(ASLLRP_CSV_FILE)
    # Rinominiamo la colonna per coerenza, se necessario
    if "Utterance video filename" in df_asllrp.columns:
        df_asllrp = df_asllrp.rename(columns={"Utterance video filename": "video_name"})

    video_files_to_process = df_asllrp["video_name"].unique().tolist()
    print(f"Trovati {len(video_files_to_process)} video unici da processare.")

    # Visualizza i primi 5 nomi di file video
    print("\nEsempio di nomi di file video:")
    for video_name in video_files_to_process[:5]:
        print(f"- {video_name}")

except FileNotFoundError:
    print(f"ERRORE: File CSV non trovato al percorso: {ASLLRP_CSV_FILE}")
    video_files_to_process = []

Trovati 630 video unici da processare.

Esempio di nomi di file video:
- 99370717.mp4
- 99395897.mp4
- 99454786.mp4
- 83594638.mp4
- 83719008.mp4


## 3. Esecuzione di MediaPipe per l'Estrazione dei Landmark

Questo è il cuore del notebook. Iteriamo su ogni video della lista e utilizziamo `mediapipe` per estrarre i landmark.

**Configurazione di MediaPipe:**

Utilizzeremo il modello olistico (`mp.solutions.holistic`), che è una soluzione potente e integrata per rilevare simultaneamente:

- **Pose landmarks**: Punti chiave del corpo.
- **Face landmarks**: Punti chiave del viso.
- **Hand landmarks**: Punti chiave delle mani.

Il processo può richiedere del tempo, ma MediaPipe è generalmente molto più veloce di OpenPose e non richiede una GPU potente. I landmark estratti verranno salvati in formato JSON, replicando la struttura di output di OpenPose per garantire la compatibilità con il nostro `LandmarkDataset`.


In [5]:
def landmarks_to_dict(landmark_list, num_landmarks, num_dimensions):
    """Converte una lista di landmark di MediaPipe in un array piatto."""
    if landmark_list is None:
        return [0.0] * (num_landmarks * num_dimensions)

    coords = []
    for landmark in landmark_list.landmark:
        coords.extend([landmark.x, landmark.y, landmark.z][:num_dimensions])
    return coords


print("Inizio del processo di estrazione dei landmark con MediaPipe...")

# Inizializza il modello olistico di MediaPipe
mp_holistic = mp.solutions.holistic

with mp_holistic.Holistic(
    static_image_mode=False,
    model_complexity=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
) as holistic:

    # Cicla su ogni file video da processare con una barra di avanzamento
    for video_filename in tqdm(video_files_to_process, desc="Estrazione Landmark"):

        # Percorso completo del video di input
        input_video_path = os.path.join(ASLLRP_VIDEOS_DIR, video_filename)

        # Crea una sottocartella specifica per i JSON di questo video
        output_json_video_dir = os.path.join(
            OUTPUT_JSON_DIR, os.path.splitext(video_filename)[0]
        )
        os.makedirs(output_json_video_dir, exist_ok=True)

        # Verifica se il video esiste
        if not os.path.exists(input_video_path):
            print(f"Video non trovato: {input_video_path}. Salto.")
            continue

        # Apri il video
        cap = cv2.VideoCapture(input_video_path)
        frame_idx = 0

        while cap.isOpened():
            success, image = cap.read()
            if not success:
                break

            # Converte l'immagine da BGR a RGB
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Processa l'immagine con MediaPipe
            results = holistic.process(image_rgb)

            # Estrai i landmark e convertili in un formato simile a OpenPose
            pose_landmarks = landmarks_to_dict(results.pose_landmarks, 33, 3)
            face_landmarks = landmarks_to_dict(results.face_landmarks, 468, 3)
            left_hand_landmarks = landmarks_to_dict(results.left_hand_landmarks, 21, 3)
            right_hand_landmarks = landmarks_to_dict(
                results.right_hand_landmarks, 21, 3
            )

            # Crea la struttura dati JSON compatibile
            output_data = {
                "version": 1.3,
                "people": [
                    {
                        "person_id": [-1],
                        "pose_keypoints_2d": pose_landmarks,  # Adattare se necessario
                        "face_keypoints_2d": face_landmarks,  # Adattare se necessario
                        "hand_left_keypoints_2d": left_hand_landmarks,
                        "hand_right_keypoints_2d": right_hand_landmarks,
                        "pose_keypoints_3d": [],
                        "face_keypoints_3d": [],
                        "hand_left_keypoints_3d": [],
                        "hand_right_keypoints_3d": [],
                    }
                ],
            }

            # Salva il file JSON per il frame corrente
            output_filename = os.path.join(
                output_json_video_dir,
                f"{os.path.splitext(video_filename)[0]}_{frame_idx:012d}_keypoints.json",
            )
            with open(output_filename, "w") as f:
                json.dump(output_data, f)

            frame_idx += 1

        cap.release()

print("\nEstrazione dei landmark completata!")

Inizio del processo di estrazione dei landmark con MediaPipe...


I0000 00:00:1759136495.473667  209697 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M3
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
Estrazione Landmark:   0%|          | 0/630 [00:00<?, ?it/s]W0000 00:00:1759136495.587989  262713 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759136495.643980  262711 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759136495.647438  262717 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759136495.649193  262714 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759136495.649621  262715 infere


Estrazione dei landmark completata!


## 4. Passaggi Successivi

Se l'esecuzione è andata a buon fine, ora dovresti avere la cartella `data/raw/ASLLRP/mediapipe_output/json` popolata con sottocartelle, una per ogni video processato, contenenti i file JSON dei landmark.

A questo punto, dobbiamo fare in modo che il nostro `LandmarkDataset` sappia dove trovare questi nuovi dati. La soluzione migliore è modificare la classe `LandmarkDataset` per accettare una **lista di directory di landmark** invece di una sola.

Modificheremo `src/data_pipeline/landmark_dataset.py` e `src/utils/training_utils.py` per supportare questa nuova configurazione.
